# Building and Running the Fraud Detection Pipeline

In this notebook, we'll build and run a Kubeflow Pipeline to train our fraud detection model.

In [2]:
import sys
import os

# Add the project root directory to Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))  # Adjust if needed
if project_root not in sys.path:
    sys.path.append(project_root)
    print(f"Added {project_root} to Python path")

Added /Users/prashanth.chaitanya/git-workspaces/kubeflow/kserve-example to Python path


In [3]:
# Import required libraries
import kfp
import pandas as pd
import os
import numpy as np

## Compile and Run the Pipeline

In [9]:
# Import our pipeline
from src.pipeline.pipeline import fraud_detection_pipeline

# Compile the pipeline
pipeline_func = fraud_detection_pipeline
pipeline_filename = "fraud_detection_pipeline.yaml"
kfp.compiler.Compiler().compile(pipeline_func, pipeline_filename)

print(f"Pipeline compiled to {pipeline_filename}")

Pipeline compiled to fraud_detection_pipeline.yaml


In [10]:
from src.client.client_manager import KFPClientManager

# initialize a KFPClientManager
kfp_client_manager = KFPClientManager(
    api_url="http://localhost:8080/pipeline",
    skip_tls_verify=True,

    dex_username="user@example.com",
    dex_password="12341234",

    # can be 'ldap' or 'local' depending on your Dex configuration
    dex_auth_type="local",
)

# get a newly authenticated KFP client
# TIP: long-lived sessions might need to get a new client when their session expires
kfp_client = kfp_client_manager.create_kfp_client()

# Create an experiment
experiment_name = "fraud-detection-kserve"
experiment = kfp_client.create_experiment(name=experiment_name, namespace="kubeflow-user-example-com")

# Submit the pipeline run
run = kfp_client.create_run_from_pipeline_package(
    experiment_id=experiment.experiment_id,
    run_name="fraud-detection-training",
    pipeline_file=pipeline_filename,
    arguments={
        "model_name": "fraud-detection",
        "model_version": "v11"
    }
)

print(f"Pipeline run submitted with ID: {run.run_id}")

/Users/prashanth.chaitanya/git-workspaces/kubeflow/kserve-example/.venv/lib/python3.11/site-packages/kfp/client/client.py:159: FutureWarning: This client only works with Kubeflow Pipeline v2.0.0-beta.2 and later versions.
  warnings.warn(


Pipeline run submitted with ID: 4f0e5fcf-f5a6-4f21-a29c-d67bb885b717


## Monitor the Pipeline Run

In [11]:
# Get pipeline run status
run_details = kfp_client.get_run(run.run_id)
print(f"Pipeline status: {run_details.state}")

Pipeline status: RUNNING


In [4]:
# Get the auth cookie using the credentials from your setup
from src.client.client_manager import KFPClientManager
manager = KFPClientManager(
    api_url='http://localhost:8080/pipeline',
    skip_tls_verify=True,
    dex_username='user@example.com',
    dex_password='12341234',
    dex_auth_type='local'
)
cookies = manager._get_session_cookies()
print(cookies)



oauth2_proxy_kubeflow=hGbpz2HTB4wVbQ41gIW_eO-uuTLbJf_VigP5II16agWt-yCrFX4Bs_uvOQJxiC15dGsmHXRS2TI0a-Lx_xfeqODUUbUOK9qt_fAze-CTpu76rTIb_5AQsQzKqLr1PvfS1KBJMQMPHd7uVVIHva1oNwiOgqSRmdF9lgLI_MS1WdHlksOBEKfwq8QFxlX7WRKA4h1ztD6FjyQD2I9SywNiIuGwcAtOen-sacHY0oGWucxHaxavYwzSW75EeGAZ4QXYvC3hJfrCLgasFNMtj1CjLF9Ny8oEvbtu88NRD4I3XhLTHzelcGQ-tAWiQTVjekop9HGCqAgWkKA_EM4L6fgq23Kgq07i_d3rH48HKaTdy7UpfFlqC1-U_E10eASAYF26O4KRTSYoBeXDGZcBtgCg-5IWDB-qg40kPeIL1Du3vFkvIgQWdhSh3gycZb4UU5NAuul-hxooXdLylx-ctvAqN1CYhqUZy-WqvP0aUrd5e0RYoukpwHaAMLNf0G5FWRFyXZInbHMiDdUa9TnM2SlxYGcBtkBaIVxDYwfQ8KUzBAu254n3P7QtZSHKShgms40h-9CJTNR2v4_9dnM7rgGUUc_gXqNgoy2-1g9eleclYYja0MA2Csd0-iOzNHiWzSFkV-sCZoGEw4exMl3Xz9vZPymh5lU_d73dKv3ZYSyXOw3plIgHScLhud49Iom2sa9ZbDlLFzJonuzVfELtXny_Zzl35jNRgOQ2rMwjTqTvqusEtOfkoY6mDZwEL6ViWckg_1qvLOV2bxVxYHMxty3F996kW377qtHlu-6mXV97p8BKJ5c3x3JTSg9b-tmogoZut0_h9rwgikTDkmyQXqVixWPSoFEtJMWTkT80Hj03uupKKLU8QVgGSa0kVpBalIYk1uh07zN-nyjNjc0ydaJ0I0f44pN2wU9JS9RqvKv3UEElXSIJ72WTNHY_a3q616vIk2BeUfr8dh1A9y6Ieu